In [35]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jan 15 12:18:14 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0              29W /  70W |   1493MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [36]:
!pip cache purge
!pip install --upgrade pip  # Upgrade pip to the latest version
!pip install -q huggingface_hub  # Install using the new package name
!pip install -q huggingface-cli
!pip install -q transformers accelerate bitsandbytes
!pip install -q torch torchvision torchaudio
!pip install -q tiktoken requests sentencepiece
!pip install -q transformers_stream_generator # Install the missing package
!pip install -q requests
!pip install -q huggingface_hub

Files removed: 2
ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli


In [37]:
from google.colab import userdata
import os
from huggingface_hub import login
# Retrieve your Hugging Face token from Colab secrets
HUGGINGFACE_TOKEN = userdata.get('HF_TOKEN')
login(HUGGINGFACE_TOKEN, add_to_git_credential=True)

#from huggingface_hub import notebook_login
#notebook_login()
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline
import torch
from transformers import BitsAndBytesConfig
#from bitsandbytes import BitsAndBytesConfig



In [38]:
QWEN = "Qwen/Qwen2.5-0.5B"

messages = [
    {"role": "system", "content": "eres un asistente util"},
    {"role": "user", "content": "Cuentame un chiste de peras"}
]


In [39]:
# Quantificacion la magia es aqui

if torch.cuda.is_available():
    # Use CUDA if available
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    device_map = "auto"  # Use GPU
else:
    # Fallback to CPU if CUDA is not available
    quantization_config = None  # Disable quantization
    device_map = "cpu"  # Use CPU

In [40]:
# TOKEN
tokenizer = AutoTokenizer.from_pretrained(QWEN)
tokenizer.pad_token = tokenizer.eos_token
# if cuda is available inputs to cuda, otherwise CPU
if torch.cuda.is_available():
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
else:
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cpu")
print(inputs)

tensor([[151644,   8948,    198,  12917,    650,    438,    380,   6817,   4094,
         151645,    198, 151644,    872,    198,     34,  11680,    373,    650,
            521,  16776,    409,    817,    300, 151645,    198]],
       device='cuda:0')


In [41]:
# MODEL descarga modelo y pesos *importante*

model = AutoModelForCausalLM.from_pretrained(QWEN, device_map=device_map, quantization_config=quantization_config)



In [42]:
memory = model.get_memory_footprint() / 1e6
print(f"Peso en Memory RAM del modelo: {memory:,.1f} MB")

Peso en Memory RAM del modelo: 451.3 MB


In [43]:
model # capa de entrada vocabulario , numeros: Embedding(151936, 896)
# tenemos capa de atencion: Qwen2SdpaAttention, multicapa Qwen2MLP, funcion de activacion: SiLU()
# capa de normalizacion:  Qwen2RMSNorm y capa linial  Linear

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    

In [44]:
outputs = model.generate( inputs, max_new_tokens=80 )
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<|im_start|>system
eres un asistente util<|im_end|>
<|im_start|>user
Cuentame un chiste de peras<|im_end|>
Cuentame un chiste de perasacco
Cuentame un chiste de perasacco
Cuentame un chiste de perasacco
Cuentame un chiste de perasacco
Cuentame un chiste de perasacco
Cuentame un chiste de perasacco
Cuentame un chiste de perasacco
Cuentame


In [45]:
# Limpiar
model.save_pretrained("qwen_quantized")
del inputs, outputs, model
torch.cuda.empty_cache()

In [46]:
# funcion de generacion y stream quantificado , recuerda que la config la tiene arriba

def generate(model, messages):
  tokenizer = AutoTokenizer.from_pretrained(QWEN)
  tokenizer.pad_token = tokenizer.eos_token

  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
  streamer = TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(QWEN, device_map="auto", quantization_config=quantization_config)
  outputs = model.generate( inputs, max_new_tokens=80, streamer=streamer )
  # Now you can save the model
  model.save_pretrained("qwen_quantized")
  del tokenizer, streamer, model, inputs , outputs
  torch.cuda.empty_cache()


In [47]:
from google.colab import files
!zip -r qwen_quantized.zip qwen_quantized
# Check if the zip file was created before attempting to download
import os
if os.path.exists("qwen_quantized.zip"):
  files.download('qwen_quantized.zip')
else:
  print("Error: qwen_quantized.zip not found. Make sure the 'generate' function was called and the model was saved successfully.")

  adding: qwen_quantized/ (stored 0%)
  adding: qwen_quantized/model.safetensors (deflated 16%)
  adding: qwen_quantized/generation_config.json (deflated 29%)
  adding: qwen_quantized/config.json (deflated 55%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>